###                                                HW9
1. Download the new test2_new.txt data which contains the ground-truth of the track ID recommendations. 

2. Use the userID in test2_new.txt to fetch the related rating scores in the training data for the same users. 

3. For the fetched rating score matrix, use the "1" and "0" recommendations in test2 to training your classifier.  You need to use 

from pyspark.ml.classification import ******
You need to try 4 different classifiers in the pyspark.ml library as shown in the lecture notes to train your models.  

4. Then apply the trained models to what you fetched rating scores from the final project test data and make predictions.  Submit your new results at Kaggle. 

#### 1.build the Gradient-Boosted Tree Classifier

In [ ]:
#pip install pyspark

In [ ]:
#conda install -c cyclus java-jdk

In [ ]:
#pip install py4j

In [1]:
#load pySpark modules 

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()
sc=spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/30 16:15:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/30 16:15:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/30 16:15:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/04/30 16:15:56 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/04/30 16:15:56 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
#load training data file into pySpark dataFrame format

training = spark.read.csv("test_new.csv", header = False)
#rename training dataFrame column names

training = training.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")
training = training.withColumn('TrackID', F.concat(F.col('userID'),F.lit('_'), F.col('itemID'))).select('TrackID','rating')
training.show(5)


+-------------+------+
|      TrackID|rating|
+-------------+------+
| 200031_30877|     1|
|  200031_8244|     1|
|200031_130183|     0|
|200031_198762|     0|
| 200031_34503|     1|
+-------------+------+
only showing top 5 rows



In [3]:
score = spark.read.csv("score.csv", header = False)
#rename training dataFrame column names

score = score.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "score1").withColumnRenamed("_c3", "score2")

score.show(5)

+------+------+------+------+
|userID|itemID|score1|score2|
+------+------+------+------+
|199810|208019|   0.0|   0.0|
|199810| 74139|   0.0|   0.0|
|199810|  9903|   0.0|   0.0|
|199810|242681|   0.0|   0.0|
|199810| 18515|   0.0|  70.0|
+------+------+------+------+
only showing top 5 rows



In [4]:

weighted_score = score.withColumn("weighted_score", F.round(0.8*score.score1+0.2*score.score2))
weighted_score = weighted_score.withColumn('TrackID', F.concat(F.col('userID'),F.lit('_'), F.col('itemID'))).select('TrackID','weighted_score')
weighted_score.show(5)

+-------------+--------------+
|      TrackID|weighted_score|
+-------------+--------------+
|199810_208019|           0.0|
| 199810_74139|           0.0|
|  199810_9903|           0.0|
|199810_242681|           0.0|
| 199810_18515|          14.0|
+-------------+--------------+
only showing top 5 rows



In [5]:
weighted_score.printSchema()

root
 |-- TrackID: string (nullable = true)
 |-- weighted_score: double (nullable = true)



In [6]:
df = training.join(weighted_score,training.TrackID == weighted_score.TrackID,"inner").drop(weighted_score.TrackID).select('TrackID','weighted_score','rating')
df.show(6)

+-------------+--------------+------+
|      TrackID|weighted_score|rating|
+-------------+--------------+------+
| 200031_30877|          82.0|     1|
|  200031_8244|          72.0|     1|
|200031_130183|           0.0|     0|
|200031_198762|           0.0|     0|
| 200031_34503|          82.0|     1|
|200031_227283|          18.0|     0|
+-------------+--------------+------+
only showing top 6 rows



In [7]:
df.printSchema()

root
 |-- TrackID: string (nullable = true)
 |-- weighted_score: double (nullable = true)
 |-- rating: string (nullable = true)



In [8]:
from pyspark.sql.types import IntegerType

df = df.withColumn("weighted_score", df["weighted_score"].cast('float'))

df.show(3)

+-------------+--------------+------+
|      TrackID|weighted_score|rating|
+-------------+--------------+------+
| 200031_30877|          82.0|     1|
|  200031_8244|          72.0|     1|
|200031_130183|           0.0|     0|
+-------------+--------------+------+
only showing top 3 rows



In [9]:
df.printSchema()

root
 |-- TrackID: string (nullable = true)
 |-- weighted_score: float (nullable = true)
 |-- rating: string (nullable = true)



### build the model

In [10]:

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorIndexer,VectorAssembler
from pyspark.ml.classification import GBTClassifier


In [25]:
# Configure an ML pipeline
label_stringIdx = StringIndexer(inputCol = 'rating', outputCol = 'label')
numericCols = ['weighted_score']
numVect = VectorAssembler(inputCols = ['weighted_score'], outputCol="features")
# Train a GBT model.
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)
pipeline = Pipeline(stages=[numVect, label_stringIdx, gbt])
# Split the data into training and test sets (30% held out for testing)
train, test = df.randomSplit([0.7, 0.3], seed = 2018)
# Fit the pipeline to training documents.
gbtmodel = pipeline.fit(train)


### Make predictions on the test set

In [26]:
predictions = gbtmodel.transform(test)
predictions.show(12)

+-------------+--------------+------+--------+-----+--------------------+--------------------+----------+
|      TrackID|weighted_score|rating|features|label|       rawPrediction|         probability|prediction|
+-------------+--------------+------+--------+-----+--------------------+--------------------+----------+
|200031_130183|           0.0|     0|   [0.0]|  0.0|[0.70770409311086...|[0.80461755545132...|       0.0|
| 200032_18681|          90.0|     1|  [90.0]|  1.0|[-1.2967852195058...|[0.06955336402159...|       1.0|
| 200032_64167|          90.0|     1|  [90.0]|  1.0|[-1.2967852195058...|[0.06955336402159...|       1.0|
|200055_134398|          18.0|     1|  [18.0]|  1.0|[-0.6430944326812...|[0.21649858344049...|       1.0|
| 200055_56695|          90.0|     1|  [90.0]|  1.0|[-1.2967852195058...|[0.06955336402159...|       1.0|
|200065_179571|           0.0|     0|   [0.0]|  0.0|[0.70770409311086...|[0.80461755545132...|       0.0|
| 200065_26875|          90.0|     1|  [90.0]|

### Evaluate our Logistic Regression model.

In [27]:
#Evaluating prediction results

tp = float(predictions.filter("prediction == 1.0 AND rating == 1").count())
fp = float(predictions.filter("prediction == 1.0 AND rating == 0").count())
tn = float(predictions.filter("prediction == 0.0 AND rating == 0").count())
fn = float(predictions.filter("prediction == 0.0 AND rating == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|             680.0|
|       FP|              67.0|
|       TN|             770.0|
|       FN|             197.0|
|Precision|0.9103078982597055|
|   Recall|0.7753705815279361|
|       F1|0.8374384236453202|
+---------+------------------+



In [28]:
#Evaluate our Logistic Regression model.
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.8688718328068018


In [29]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.131128


### Make predictions on the rating scores data

In [30]:
#Use the model to pedict the whole test data
prediction_all = gbtmodel.transform(weighted_score)
prediction_all.show(12)

+-------------+--------------+--------+--------------------+--------------------+----------+
|      TrackID|weighted_score|features|       rawPrediction|         probability|prediction|
+-------------+--------------+--------+--------------------+--------------------+----------+
|199810_208019|           0.0|   [0.0]|[0.70770409311086...|[0.80461755545132...|       0.0|
| 199810_74139|           0.0|   [0.0]|[0.70770409311086...|[0.80461755545132...|       0.0|
|  199810_9903|           0.0|   [0.0]|[0.70770409311086...|[0.80461755545132...|       0.0|
|199810_242681|           0.0|   [0.0]|[0.70770409311086...|[0.80461755545132...|       0.0|
| 199810_18515|          14.0|  [14.0]|[-0.2645640406367...|[0.37072027218164...|       1.0|
|199810_105760|          18.0|  [18.0]|[-0.6430944326812...|[0.21649858344049...|       1.0|
|199812_276940|           0.0|   [0.0]|[0.70770409311086...|[0.80461755545132...|       0.0|
|199812_142408|         100.0| [100.0]|[-1.3259026792203...|[0.0658778

22/04/30 16:28:05 WARN StringIndexerModel: Input column rating does not exist during transformation. Skip StringIndexerModel for this column.


In [31]:
#select "TrackID","Predictor"
prediction_All = prediction_all.withColumnRenamed("prediction", "Predictor").select("TrackID","Predictor")

In [32]:
prediction_All.show(5)

+-------------+---------+
|      TrackID|Predictor|
+-------------+---------+
|199810_208019|      0.0|
| 199810_74139|      0.0|
|  199810_9903|      0.0|
|199810_242681|      0.0|
| 199810_18515|      1.0|
+-------------+---------+
only showing top 5 rows



In [37]:
prediction_All.count()

120000

In [38]:
#save the results to csv file
prediction_All.toPandas().to_csv('prediction_gbt.csv')